# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [2]:
pip install python-dotenv


In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())



In [2]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [3]:
import pandas as pd

data = {
    'table': ['Customers', 'Orders', 'Products'],
    'definition': [
        'Contains customer information like name, email, and phone number.',
        'Contains order details including order ID, customer ID, and total price.',
        'Contains product details such as product name, category, and price.'
    ]
}

df = pd.DataFrame(data)
print(df)


       table                                         definition
0  Customers  Contains customer information like name, email...
1     Orders  Contains order details including order ID, cus...
2   Products  Contains product details such as product name,...


In [4]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [5]:
print(text_tables)

Customers: Contains customer information like name, email, and phone number.
Orders: Contains order details including order ID, customer ID, and total price.
Products: Contains product details such as product name, category, and price.


In [6]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [7]:
# #Creating the prompt, with the user questions and the tables definitions.
# pqt1 = prompt_question_tables.format(tables=text_tables, question=#ENTER YOUR QUERY HERE)

#                                      # Creating the prompt, with the user questions and the tables definitions.
user_query = "Which customers have placed orders in the last 30 days?"

pqt1 = prompt_question_tables.format(tables=text_tables, question=user_query)

print(pqt1)



Given the following tables and their content definitions,
###Tables
Customers: Contains customer information like name, email, and phone number.
Orders: Contains order details including order ID, customer ID, and total price.
Products: Contains product details such as product name, category, and price.

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
Which customers have placed orders in the last 30 days?



In [8]:

print(return_OAI(pqt1))

{
  "Tables": ["Customers", "Orders"]
}


In [10]:
# Define the user query
user_query = "Which products have the highest sales in the last 6 months?"

# Format the prompt with the actual query
pqt3 = prompt_question_tables.format(tables=text_tables, question=user_query)

print(pqt3)  # To verify the formatted prompt



Given the following tables and their content definitions,
###Tables
Customers: Contains customer information like name, email, and phone number.
Orders: Contains order details including order ID, customer ID, and total price.
Products: Contains product details such as product name, category, and price.

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
Which products have the highest sales in the last 6 months?



In [11]:
print(return_OAI(pqt3))

{
  "Tables": ["Orders", "Products"]
}


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?